<a href="https://colab.research.google.com/github/ClementD007/Learning/blob/main/Web%20Scraper%20101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint


pages = np.arange(1, 1001, 50)
headers = {"Accept-Language": "en-US, en;q=0.5"}

titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

for page in pages:

  page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt", headers=headers)

  soup = BeautifulSoup(page.text, 'html.parser')
  
  movie_div = soup.find_all('div', class_='lister-item mode-advanced')

  sleep(randint(2,10))

  for container in movie_div:
    #name
    name = container.h3.a.text
    titles.append(name)
    #year
    year = container.h3.find('span', class_='lister-item-year').text
    years.append(year)
    #runtime
    runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else ''
    time.append(runtime)
    #rating
    imdb = float(container.strong.text)
    imdb_ratings.append(imdb)
    #metascore
    m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
    metascores.append(m_score)
    #votes & gross
    nv = container.find_all('span', attrs={'name': 'nv'})   
    vote = nv[0].text
    votes.append(vote)    
    grosses = nv[1].text if len(nv) > 1 else ''
    us_gross.append(grosses) 

movies = pd.DataFrame({
'movie': titles,
'year': years,
'timeMin': time,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'us_grossMillions': us_gross,
})


movies['votes'] = movies['votes'].str.replace(',', '').astype(int)

movies.loc[:, 'year'] = movies['year'].str[-5:-1].astype(int)

movies['timeMin'] = movies['timeMin'].astype(str)
movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)

movies['metascore'] = movies['metascore'].str.extract('(\d+)')
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')



In [5]:
movies.head()

,movie,year,timeMin,imdb,metascore,votes,us_grossMillions
0,The Trial of the Chicago 7,2020,129,7.9,76.0,57100,NaN
1,Tenet,2020,150,7.8,69.0,145656,53.80
2,Spider-Man: Into the Spider-Verse,2018,117,8.4,87.0,352823,190.24
3,The Gentlemen,2019,113,7.8,51.0,204968,NaN
4,Harry Potter and the Sorcerer's Stone,2001,152,7.6,64.0,641548,317.58


In [9]:
movies.to_excel('Recents\movies.xlsx')